In [5]:
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium 
import geopy 

In [17]:
class Current_locations:
    
    def __init__(self, directory_name):
        
        self.dict_name = directory_name
        self.list_of_files = os.listdir(os.path.join(os.getcwd(), directory_name))
        self.num_of_files = len(self.list_of_files)
        
        assert(self.num_of_files > 1)
        
        #create pandas data_frame for first file
        file_name = self.list_of_files[0]
        path = os.path.join(os.getcwd(), directory_name, file_name)
        f = open(path)
        json_data = json.load(f)
        self.data = pd.DataFrame.from_dict(json_data["result"])
        
        #create main pandas dataframe
        for i in range(1, self.num_of_files):
            self.__concatenate_data(i)
        self.data['Time'] = pd.to_datetime(self.data['Time'])

        
        #create list with all vehicle_numbers
        self.all_vehicle_numbers = self.data["VehicleNumber"].to_numpy()
        self.all_vehicle_numbers = np.unique(self.all_vehicle_numbers)
        
    
    def get_data(self):
        return self.data
    
    ##1. Functions for data preparation:
    def __concatenate_data(self, i):
        file_name = self.list_of_files[i]
        path = os.path.join(os.getcwd(), self.dict_name, file_name)
        f = open(path)
        json_data = json.load(f)
        data_i = pd.DataFrame.from_dict(json_data["result"])
        self.data = pd.concat([self.data, data_i], axis=0)
        
    def get_start_time_of_downloading(self):
        times = np.copy(self.list_of_files)
        for i in range(self.num_of_files):
            times[i] = times[i][:-5].split("_")[1].replace(',', '')
        times = times.astype("datetime64[ns]")
        return np.min(times)
    
    def get_all_vehicle_numbers(self):
        return self.all_vehicle_numbers
    
    ##2. 
    def analysis_for_one_bus(self, bus_id):
        min_time = self.get_start_time_of_downloading() - np.timedelta64(60,'s')
        bus = self.data[self.data["VehicleNumber"] == bus_id]
        bus = bus.sort_values("Time")
        bus_good_time = bus[bus["Time"] > min_time]
        ##ans[bus_table, n_rows, ]
        ans = ['', '', '']
        return bus_good_time.to_numpy()

In [18]:
to_analyse = Current_locations("2024-02-20, 12:38 ztm")
to_analyse.get_start_time_of_downloading()

numpy.datetime64('2024-02-20T12:38:30.000000000')

In [19]:
to_analyse.get_all_vehicle_numbers()

array(['1000', '1001', '1002', ..., '9953', '9954', '9955'], dtype=object)

In [16]:
def rad_to_deg(radians):
    degrees = radians * 180 / np.pi
    return degrees

def deg_to_rad(degrees):
    radians = degrees * np.pi / 180
    return radians

#https://pl.martech.zone/calculate-great-circle-distance/
def getDistanceBetweenPointsNew(latitude1, longitude1, latitude2, longitude2):
    
    theta = longitude1 - longitude2
    
    distance = 60 * 1.1515 * rad_to_deg(
        np.arccos(
            (np.sin(deg_to_rad(latitude1)) * np.sin(deg_to_rad(latitude2))) + 
            (np.cos(deg_to_rad(latitude1)) * np.cos(deg_to_rad(latitude2)) * np.cos(deg_to_rad(theta)))
        )
    )
    
    return round(distance * 1.609344, 2) #distance in kilometers

In [32]:
my_data = to_analyse.analysis_for_one_bus('9954')

## my_data[i][Lines, Lon, VehicleNumber, Time, Lat, Brigade]
len(my_data)
if (len(my_data) > 1):
    print("ok")
for i in range(len(my_data) - 1):
    print(f"{my_data[i][3]}, .. {my_data[i + 1][3]}")
    if my_data[i][3] < my_data[i + 1][3]:
        time = (my_data[i + 1][3] - my_data[i][3]).seconds
        lat1 = my_data[i][4]
        lon1 = my_data[i][1]
        lat2 = my_data[i + 1][4]
        lon2 = my_data[i + 1][1]
        speed = getDistanceBetweenPointsNew(lat1, lon1, lat2, lon2) / (time / 3600)
        print(f", lat1:{lat1}, lon1:{lon1}, dist: {getDistanceBetweenPointsNew(lat1, lon1, lat2, lon2)}, \
time: {time}, speed: {speed}")
    print()

ok
2024-02-20 12:38:18, .. 2024-02-20 12:39:36
, lat1:52.217258, lon1:20.960697, dist: 0.45, time: 78, speed: 20.76923076923077

2024-02-20 12:39:36, .. 2024-02-20 12:40:33
, lat1:52.22121, lon1:20.959654, dist: 0.37, time: 57, speed: 23.368421052631575

2024-02-20 12:40:33, .. 2024-02-20 12:41:39
, lat1:52.224384, lon1:20.958181, dist: 0.39, time: 66, speed: 21.272727272727273

2024-02-20 12:41:39, .. 2024-02-20 12:42:47
, lat1:52.227848, lon1:20.9575, dist: 0.43, time: 68, speed: 22.764705882352942

2024-02-20 12:42:47, .. 2024-02-20 12:43:45
, lat1:52.231682, lon1:20.956701, dist: 0.15, time: 58, speed: 9.310344827586206

2024-02-20 12:43:45, .. 2024-02-20 12:44:50
, lat1:52.232998, lon1:20.956215, dist: 0.62, time: 65, speed: 34.338461538461544

2024-02-20 12:44:50, .. 2024-02-20 12:45:46
, lat1:52.238537, lon1:20.956589, dist: 0.48, time: 56, speed: 30.857142857142858

2024-02-20 12:45:46, .. 2024-02-20 12:46:43
, lat1:52.242767, lon1:20.958094, dist: 0.44, time: 57, speed: 27.789

In [24]:
t_a = Current_locations("2024-02-20, 19:50 ztm")
#t_a.get_data()

In [1]:
import math

class Ulamek:
     
    def __init__(self, licznik, mianownik):
        assert mianownik != 0
        self.l = licznik // math.gcd(licznik, mianownik)
        self.m = mianownik // math.gcd(licznik, mianownik)
        
    def __add__(self, inny):
        l2 = inny.l
        m2 = inny.m
        
        wynikl = self.l * m2 + self.m * l2
        wynikm = self.m * m2

        return Ulamek(wynikl // math.gcd(wynikl, wynikm), wynikm // math.gcd(wynikl, wynikm))    
        
    def __sub__(self, inny):
        l2 = inny.l
        m2 = inny.m
        
        wynikl = self.l * m2 - self.m * l2
        wynikm = self.m * m2

        return Ulamek(wynikl // math.gcd(wynikl, wynikm), wynikm // math.gcd(wynikl, wynikm))
    
    def __mul__(self, inny):
        l2 = inny.l
        m2 = inny.m
        wynikl = self.l * l2
        wynikm = self.m * m2 
        return Ulamek(wynikl // math.gcd(wynikl, wynikm), wynikm // math.gcd(wynikl, wynikm))
    
    def __truediv__(self, inny):
        l2 = inny.l
        m2 = inny.m
        assert(l2 != 0)
        wynikl = self.l * m2
        wynikm = self.m * l2
        return Ulamek(wynikl // math.gcd(wynikl, wynikm), wynikm // math.gcd(wynikl, wynikm))
        
        
    def __str__(self): ##wypisz liczbe
        if self.ujemny() == True:
            if self.modul(self.m) != 1:
                return f"- {self.modul(int(self.l))} / {self.modul(int(self.m))}"
            else:
                return f"- {self.modul(int(self.l))}"   
        else:
            if self.m != 1:
                return f"{int(self.l)} / {int(self.m)}"
            else:
                return f"{int(self.l)}"
    
    def ujemny(self):
        if self.l < 0 and self.m > 0:
            return True
        if self.l > 0 and self.m < 0:
            return True
        return False
    
    def modul(self, a):
        if a > 0:
            return a
        else:
            return -a
        

In [2]:
def zrob_ul(a, b):
    return Ulamek(a, b)

U1 = zrob_ul(4, 2)
U1.__str__()

'2'